# Customzie Formulation

Encapsuled the optimization problem calss, AMS provides direct access to the optimization formulation, where users have the option to customize the formulation without playing with the source code.

In this example, we will walk through the optimization problem structure and show how to customize the formulation.

In [1]:
import numpy as np

import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams:{ams.__version__}')

Last run time: 2024-02-20 10:49:56
ams:0.8.5.post44.dev0+gf8df7b6


In [3]:
ams.config_logger(stream_level=20)

## Inspect the Optimization Problem Structure

In [4]:
sp = ams.load(ams.get_case('5bus/pjm5bus_demo.xlsx'),
                  setup=True,
                  no_output=True,)

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/5bus/pjm5bus_demo.xlsx"...
Input file parsed in 0.1247 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
If expect a line outage, please set 'u' to 0.
System set up in 0.0020 seconds.


In AMS, a routine collects the descriptive dispatch formulations.
`DCOPF`, `RTED`, etc, are the subclasses of `RoutineBase`.

In [5]:
sp.DCOPF.init()

Routine <DCOPF> initialized in 0.0088 seconds.


True

After successful initialization, the attribute `om` is populated with CVXPY-based optimization problem.

The user can even hack to the source `prob` attribute to customize it if necessary.

In [6]:
type(sp.DCOPF.om.prob)

cvxpy.problems.problem.Problem

## Customize Built-in Formulation

To maintain the consistency, AMS provides a set of built-in formulation customization methods to streamline the customization process.

Here we extend DCOPF with consideration of CO2 emission.
To simplify the demonstration, following assumptions are made:
1. Variable `eg` is the CO2 emission of each generator. It is proportional to the generation, described by a parameter `ke` in the unit t/p.u..
1. Total CO2 emmission is limited by a constant cap `te`, in the unit $t$.
1. A tax `ce` is imposed on each unit of CO2 emission in the unit of $/p.u., and the tax is included in the objective function.

### Add services

Services are used to store values or build matrix for easier formulation.

In [7]:
sp.DCOPF.addService(name='te', tex_name='t_e',
                    unit='t', info='emission cap',
                    value=12)

True

In [8]:
sp.DCOPF.te

ValueService: DCOPF.te

### Add parameters

We need the following parameters to be defined as `RParam`: `ke` and `ce`. They should be 1D array in the same length as the number of generators and `te` is a scalar.

For a general `RParam`, it has attributes `model`, `indexer`, and `imodel` to describe its source model and index model. The definition of `c2` in DCOPF source code is a good example.
However, for ones defined through API, since there is no model containing it, all above attributes are not applicable, and the user should be aware of the sequence of the parameters.

Considering the sequence can be indexed by the generator index, it is used to reference the variables order.
Assuming `ke` is reciprocal to the generator capacity, and `ce` is the same for each generator, we can define the parameters as follows:

In [9]:
# get the generator indices
stg_idx = sp.DCOPF.pg.get_idx()

# get the value of pmax
pmax = sp.DCOPF.get(src='pmax', attr='v', idx=stg_idx)

# assume the emission factor is 1 for all generators
ke = np.ones_like(pmax)

# assume tax is reciprocal of pmax
ce = np.reciprocal(pmax)

In [10]:
sp.DCOPF.addRParam(name='ke', tex_name='k_e',
                   info='gen emission factor',
                   model=None, src=None, unit=None,
                   v=ke)

sp.DCOPF.addRParam(name='ce', tex_name='c_e',
                   info='gen emission tax',
                   model=None, src=None, unit=None,
                   v=ce)

True

### Add constraints

The CO2 emission cap is a simple linear inequality constraint.

If wish to revise an existing built-in constraint, you can redefine the constraint `e_str` attribute.

In [11]:
sp.DCOPF.addConstrs(name='eub', info='emission upper bound',
                    e_str='sum(mul(ke, pg)) - te', type='uq',)

True

### Revise the objective function

The `e_str` can be revised to include the CO2 emission tax.
Here we only need to append the tax term to the original objective function.

In [12]:
sp.DCOPF.obj.e_str += '+ sum(mul(ce, pg))'

### Finalize the Customization

After revising the problem, remember to initialize it before solving.

In [13]:
sp.DCOPF.init()

Routine <DCOPF> initialized in 0.0057 seconds.


True

### Solve it and Check the Results

In [14]:
sp.DCOPF.run(solver='ECOS')

DCOPF solved as optimal in 0.0110 seconds, converged after 9 iterations using solver ECOS.


True

Inspect the results.

In [15]:
sp.DCOPF.pg.v

array([0.2, 5.2, 4.4, 0.2])

In [16]:
sp.DCOPF.obj.v

5.297571428627165

Load the original problem as a baseline for comparison.

In [17]:
sp0 = ams.load(ams.get_case('5bus/pjm5bus_demo.xlsx'),
                 setup=True,
                 no_output=True,)

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/5bus/pjm5bus_demo.xlsx"...
Input file parsed in 0.0367 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
If expect a line outage, please set 'u' to 0.
System set up in 0.0021 seconds.


In [18]:
sp0.DCOPF.run(solver='ECOS')

Routine <DCOPF> initialized in 0.0066 seconds.
DCOPF solved as optimal in 0.0098 seconds, converged after 9 iterations using solver ECOS.


True

From the comparasion, we can see that the generation schedule changes.

In [19]:
sp0.DCOPF.pg.v

array([2.1, 5.2, 0.7, 2. ])

In [20]:
sp0.DCOPF.obj.v

2.3445000004668826